In [1]:
import platform as python_platform

class Platform:
    WINDOWS = "Windows"
    MACOS = "Darwin"

platform = python_platform.system()

windows_root_directory = "D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling"
mac_root_directory = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/DataCrawling"

root_directory = windows_root_directory if platform == Platform.WINDOWS else mac_root_directory

In [2]:
import os
from os.path import join, getsize
import re

import sys
sys.path.append(root_directory)
import random

from bs4 import BeautifulSoup

import file_utils as fu

In [3]:
def copy_folder_structure(source, destination):
    '''Copy the structure of the source directory to the destination directory'''
    for root, dirs, files in os.walk(source):
        for dir in dirs:
            new_dir = f"{destination}/{root.replace(source, '')}/{dir}"
            
            regex_pattern = r"\/{2,}"            
            new_dir = re.sub(regex_pattern, "/", new_dir)

            os.makedirs(new_dir, exist_ok=True)

In [4]:
song_ids_of_song_parts_dir = fu.relative_to_absolute_path(
    "DataPreprocessing/song_id_of_song_parts",
    root_path=root_directory
)

raw_html_dir = fu.relative_to_absolute_path(
    "GetAllSongHTML/song_list/raw_html",
    root_path=root_directory
)

copy_folder_structure(
    source=raw_html_dir, 
    destination=song_ids_of_song_parts_dir
)

In [5]:
def extract_metrics_and_conclusion_from_html(html_raw_content):
    '''
        Trích xuất thông tin về metrics và các kết luận từ nội dung HTML của một bài hát
    '''
    metrics = {}
    metrics_conclusion = ""
    key_conclusion = ""

    soup = BeautifulSoup(html_raw_content, "html.parser")

    # Tìm thẻ h3 có thuộc tính text chứa chuỗi ký tự, trong đó có chuỗi ký tự "Chord And Melody Metrics" bằng CSS Selector
    try:
        h3 = soup.find(
            "h3",
            string=lambda text: "Chord And Melody Metrics" in text    
        )

        # Lấy các thẻ div chứa thông tin về metrics
        metric_divs = h3.find_next_sibling("div").find_all("div")

        for div in metric_divs:
            h5 = div.find("h5")
            h2 = div.find("h2")

            metrics[h5.text] = h2.text

        metrics_conclusion_p = h3.find_next_sibling("p")
        metrics_conclusion = metrics_conclusion_p.text
        
        try:
            key_conclusion_p = h3.parent.parent.find_next_sibling("div").find("p")
            key_conclusion = key_conclusion_p.text
        except:
            print("Ignore data message: No key conclusion")
            pass
    except:
        print("Ignore data message: No metrics and conclusion found in HTML content")
        pass

    return metrics, metrics_conclusion, key_conclusion

In [6]:
def extract_all_a_hrefs_from_html(html_raw_content, text_of_a_tag="Open In Hookpad"):
    '''
        Trả về href của các thẻ a có text thoả text_of_a_tag là chuỗi ký tự con
    '''
    # Ví dụ:
    # <h2 class="margin-0">
    #     <a class="" href="https://hookpad.hooktheory.com?idOfSong=JkmZRne-mqn&amp;enableYouTube=false&amp;openBandEditorOnInit=true">
    #         Open In Hookpad
    #     </a>
    # </h2>

    soup = BeautifulSoup(html_raw_content, "html.parser")
    
    a_tags = soup.find_all("a", string=text_of_a_tag)

    hrefs = [a.get("href") for a in a_tags]

    return hrefs

In [7]:
def extract_song_part_ids_from_hrefs(open_in_hookpad_hrefs):
    '''
        Trả về song part id từ href của thẻ a "Open In Hookpad"
    '''
    # Ví dụ về href: https://hookpad.hooktheory.com?idOfSong=Weglnna-orY&amp;enableYouTube=false&amp;openBandEditorOnInit=true

    song_part_ids = []
    
    for href in open_in_hookpad_hrefs:
        song_part_id = ""

        # Tìm chuỗi "idOfSong=" trong href
        idOfSong_index = href.find("idOfSong=")
        if idOfSong_index != -1:
            # Tìm chuỗi "idOfSong=" và lấy phần sau nó
            song_part_id = href[idOfSong_index + len("idOfSong="):]

            # Tìm chuỗi "&" trong phần sau "idOfSong=" và lấy phần trước nó
            and_index = song_part_id.find("&")
            if and_index != -1:
                song_part_id = song_part_id[:and_index]
            else:
                print(f"extract_song_part_ids_from_hrefs: Cannot find '&' in {href}")
        else:
            print(f"extract_song_part_ids_from_hrefs: Cannot find 'idOfSong=' in {href}")

        if song_part_id != "":
            song_part_ids.append(song_part_id)
        else:
            print(f"extract_song_part_ids_from_hrefs: Cannot extract song part id from {href}")

    return song_part_ids

print(extract_song_part_ids_from_hrefs(
    ["https://hookpad.hooktheory.com?idOfSong=Weglnna-orY&amp;enableYouTube=false&amp;openBandEditorOnInit=true"]
))

['Weglnna-orY']


In [8]:
def extract_song_parts_ids(html_raw_content):
    '''
        Trả về song part id từ nội dung HTML của một bài hát
    '''
    text_of_a_tag = "Open In Hookpad"
    
    hrefs = extract_all_a_hrefs_from_html(
        html_raw_content, 
        text_of_a_tag
    )
    
    song_part_ids = extract_song_part_ids_from_hrefs(hrefs)

    return song_part_ids

In [9]:
def get_song_json_list_by_alphabet(letter: str):
    '''
        Trả về danh sách các bài hát có tên nghệ sĩ bắt đầu bằng chữ cái letter trong thư mục song_list_link_by_artist/json
    '''

    song_list_link_by_artist_json_dir = fu.relative_to_absolute_path(
        "GetAllSongHTML/song_list_link_by_artist/json",
        root_path=root_directory
    )

    song_json_file_by_letter = f"{song_list_link_by_artist_json_dir}/{letter.lower()}.json"

    song_json_items = fu.read_data_from_json_file(song_json_file_by_letter)

    return song_json_items

In [10]:
def song_link_to_raw_html_file_path(song_link: str):
    relative_file_path = fu.song_link_to_relative_html_file_path(song_link)

    absolute_file_path = fu.relative_to_absolute_path(
        f"GetAllSongHTML/song_list/raw_html/{relative_file_path}",
        root_path=root_directory
    )

    return absolute_file_path

In [11]:
def extract_song_parts_ids_by_letter_to_file(letter: str):
    '''
        Trích xuất danh sách các song part id của các bài hát có tên nghệ sĩ bắt đầu bằng chữ cái letter vào thư mục DataPreprocessing/song_id_of_song_parts
    '''
    song_json_list = get_song_json_list_by_alphabet(letter)

    for song_json_item in song_json_list:
        song_url = song_json_item["link"]
        print(f"-----> Extracting {song_url}")

        raw_html_path = song_link_to_raw_html_file_path(song_url)
        print(f"-----> Raw html path: {raw_html_path}")

        html_raw_content = fu.read_data_from_html_file(raw_html_path)

        metrics, metrics_conclusion, key_conclusion = extract_metrics_and_conclusion_from_html(
            html_raw_content
        )


        song_ids_of_song_parts = extract_song_parts_ids(html_raw_content)


        extracted_data = {
            "artist_name": song_json_item["artist_name"],
            "artist_link": song_json_item["artist_link"],
            "song_name": song_json_item["song_name"],
            "link": song_json_item["link"],
            "metrics": metrics,
            "metrics_conclusion": metrics_conclusion,
            "key_conclusion": key_conclusion,
            "song_parts_ids": song_ids_of_song_parts
        }

        relative_json_file_path = f"{fu.song_link_to_relative_html_file_path(song_url).split('.')[0]}.json"

        absolute_json_file_path = fu.relative_to_absolute_path(
            relative_json_file_path, root_path=song_ids_of_song_parts_dir
        )

        # DEBUG CODE START

        # # Nếu file absolute_json_file_path tồn tại và file size > 500 bytes thì bỏ qua
        # if os.path.exists(absolute_json_file_path) and os.path.getsize(absolute_json_file_path) > 50:
        #     continue
        #     print(f"-----> OK {song_json_item['link']}")
        #     print()
        # else:
        #     print(f"FAILED: {absolute_json_file_path}")
        #     # url & raw_html_path
        #     print(f"-----> {song_url}")
        #     print(f"-----> {raw_html_path}")
        #     print()
        #     continue
        
        # DEBUG CODE END
        
        fu.write_data_to_json_file(
            extracted_data,
            absolute_json_file_path
        )

        print(f"-----> Extracted {song_json_item['link']}")
        print(extracted_data)
        print(f"-----> Wrote to {absolute_json_file_path}")
        print(os.path.exists(absolute_json_file_path))
        print()

In [12]:
start_letter = "a"
end_letter = "z"

for letter in range(ord(start_letter), ord(end_letter) + 1):
    print(f"Extracting song parts ids by letter {chr(letter)}")
    extract_song_parts_ids_by_letter_to_file(chr(letter).lower())

Extracting song parts ids by letter f
-----> Extracting https://www.hooktheory.com/theorytab/view/f%28x%29/red-light
-----> Raw html path: D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling/GetAllSongHTML/song_list/raw_html/f/f%28x%29/red-light.html
DEBUG
DEBUG
-----> Extracted https://www.hooktheory.com/theorytab/view/f%28x%29/red-light
{'artist_name': 'f(x)', 'artist_link': 'https://www.hooktheory.com/theorytab/artists/f/f%2528x%2529', 'song_name': 'Red Light', 'link': 'https://www.hooktheory.com/theorytab/view/f%28x%29/red-light', 'metrics': {'Chord Complexity': '54', 'Melodic Complexity': '92', 'Chord-Melody Tension': '8', 'Chord Progression Novelty': '88', 'Chord Bass Melody': '88'}, 'metrics_conclusion': ' In terms of chords and melody, Red Light is more complex than the typical song, having above average scores in Chord Complexity, Melodic Complexity, Chord Progression Novelty and Chord-Bass Melody. ', 'key_conclusion': ' Red Light is written in the key of D Minor.

KeyboardInterrupt: 